## Practice Problems, Session 3

The ArcGIS API For Python

<h2> Problem 1 </h2>

Let's upload some additional data to our AGOL accounts.

Included within this folder is a zipped shapefile called: "wc_watershed"

This shapefile contains watershed information for walnut creek, ca. Let's upload it to our AGOL accounts as a feature layer.

In [ ]:
import os
import arcgis

In [ ]:
# sign into AGOL through python with our pro credentials
mygis = arcgis.GIS('pro')

# get a path to the zip file using os
project_path = os.getcwd()
wc_watershed_path = os.path.join(project_path, 'wc_watershed.zip')

# create the item property dictionary
item_properties = {"type": "Shapefile",
                   "title" : "Walnut Creek Watershed",
                    "tags": "Watershed, Walnut Creek, CA",
                   "description" : "Watershed in Walnut Creek, CA. Created 2022 from Python",
                   "commentsEnabled" : False}

# upload the shapefile to out AGOL account
wc_watershed_item_shp = mygis.content.add(item_properties, data=wc_watershed_path)

# publish the uploaded shapefile as a feature service feature layer
wc_watershed_feature_layer = wc_watershed_item_shp.publish()

# let's delete the uploaded shapefile, since we now have it published as a feature layer
wc_watershed_item_shp.delete()

<h2> Problem 2 </h2>

Great, now that we have our feature layer uploaded, lets create a map to visualize the 
data.

The map should be centered on walnut creek and have a zoom level of 14.

Look into the documentation for the map object. Let's change the default basemap to a different one. For this map, lets change the basemap to the "national-geographic" option.

Then, let's add the layer we just uploaded to AGOL to the map!

Hint: <a href='https://developers.arcgis.com/python/guide/using-the-map-widget/'>Here's a link to the docs regarding the map widget</a>

In [ ]:
# create map object
watershed_map = mygis.map("Walnut Creek, CA", zoomlevel=14)

#set basemap to national geographic
watershed_map.basemap = 'national-geographic'

# add our new feature layer
watershed_map.add_layer(wc_watershed_feature_layer)

# print the map out to the notebook
watershed_map

<h2> Problem 3 </h2>

Challenge Problem. This problem is a little difficult, so do not get discouraged. Loop through the feature set, use print statements,
and test out your results. Try to get to the answer incrementally.

Review the feature layer's table that we just uploaded. Notice that there are some typo's within the trail name column.

Lets clean those up using a feature set. (remember, this is similair to using an arcpy update cursor)

We will want to replace wherever there is a '*' or a '%' with an empty string: ''

In [ ]:
# log onto AGOL using your credentials using your browser. 
# navigate to your recently uploaded feature layer and copy the feature service url.
# remember to include a '/0' after the path to get the first layer (we know there is only one)

# Create a variable of the path to your rest service
wc_watershed_url = r'https://services.arcgis.com/JbibFpsuaEQr3FFG/arcgis/rest/services/wc_watershed/FeatureServer/0'

# use the FeatureLayer function to create a feature layer object using the above url
fl = arcgis.features.FeatureLayer(wc_watershed_url)

# use the query method with no input parameters to create a feature set from the above feature layer
fs = fl.query()

# loop though the feature set to start cleaning the text
for row in fs:
    # get the value of what's within the name column as a variable
    name_value = row.attributes['NAME']
    # check if % is within the name value
    if '%' in name_value:
        # if it is, lets use the replace function to remove it and replace it with an empty string
        new_name = name_value.replace('%', '')
        # reset the row.attributes['NAME'] value with the new name we just created
        row.attributes['NAME'] = new_name
    #repeat the same for '*'
    if '*' in name_value:
        new_name = name_value.replace('*', '')
        row.attributes['NAME'] = new_name

# update feature layer object
fl.edit_features(updates=fs)